# word2vec implementation

## Overview
<img src="image/word2vec_cbow_mechanism.png" align="left"/>

---
# Setups

In [1]:
import sys
import os
import re
from itertools import islice
from typing import Dict, List
import numpy as np
import tensorflow as tf

np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(linewidth=400) 

## Setup for Google Colab environment

In [2]:
try:
    import google.colab
    IN_GOOGLE_COLAB = True
except:
    IN_GOOGLE_COLAB = False
    
if IN_GOOGLE_COLAB:
    !pip install line_profiler
    !google.colab.drive.mount('/content/gdrive')
    !rm -rf /content/github
    !mkdir -p /content/github
    !git clone https://github.com/oonisim/python-programs.git /content/github
        
    import sys
    sys.path.append('/content/github/nlp/src')

## Jupyter notebook setups

In [3]:
%load_ext line_profiler
%load_ext autoreload

## Utilites

In [4]:
%autoreload 2

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

import function.fileio as fileio

# Constants

In [5]:
USE_PTB = True
DEBUG = False
VALIDATION = True

---

# Data
## Corpus

In [6]:
corpus = "To be, or not to be, that is the question that matters"
_file = "ptb.train.txt"
if USE_PTB:
    if not fileio.Function.is_file(f"~/.keras/datasets/{_file}"):
        path_to_ptb = tf.keras.utils.get_file(
            _file, 
            f'https://raw.githubusercontent.com/tomsercu/lstm/master/data/{_file}'
        )
    corpus = fileio.Function.read_file(path_to_ptb)

In [7]:
examples = corpus.split('\n')[:5]
for line in examples:
    print(line)

 aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memotec mlx nahb punts rake regatta rubens sim snack-food ssangyong swapo wachter 
 pierre <unk> N years old will join the board as a nonexecutive director nov. N 
 mr. <unk> is chairman of <unk> n.v. the dutch publishing group 
 rudolph <unk> N years old and former chairman of consolidated gold fields plc was named a nonexecutive director of this british industrial conglomerate 
 a form of asbestos once used to make kent cigarette filters has caused a high percentage of cancer deaths among a group of workers exposed to it more than N years ago researchers reported 


---
# Word indexing

In [8]:
%autoreload 2
from layer.preprocessing import (
    EventIndexing, 
    EventContext
)

## EventIndexing instance for the corpus

Adapt to the ```corpus``` and provides:
* event_to_index dictionary
* vocaburary of the corpus
* word occurrence probabilites

In [9]:
word_indexing = EventIndexing(
    name="word_indexing_on_ptb",
    corpus=corpus
)

In [11]:
words = word_indexing.list_events(range(10))
print(f"EventIndexing.vocabulary[10]:\n{words}\n")

indices = word_indexing.list_event_indices(words)
print(f"EventIndexing.event_to_index[10]:")
for item in zip(words, indices):
    print(item)

probabilities = word_indexing.list_probabilities(words)
print(f"\nEventIndexing.probabilities[10]:")
for word, p in zip(words, probabilities):
    print(f"{word:20s} : {p:.5e}")

EventIndexing.vocabulary[10]:
['<nil>' '<unk>' 'has' 'imposed' 'alter' 'subsidiary' 'bleeding' 'historically' 'asia' 'phil']

EventIndexing.event_to_index[10]:
('<nil>', 0)
('<unk>', 1)
('has', 2)
('imposed', 3)
('alter', 4)
('subsidiary', 5)
('bleeding', 6)
('historically', 7)
('asia', 8)
('phil', 9)

EventIndexing.probabilities[10]:
<nil>                : 0.00000e+00
<unk>                : 1.65308e-02
has                  : 2.43071e-03
imposed              : 6.51042e-05
alter                : 4.47793e-05
subsidiary           : 2.46237e-04
bleeding             : 2.05047e-05
historically         : 4.27888e-05
asia                 : 8.66619e-05
phil                 : 2.54424e-05


## Sentence to Sequence

In [12]:
# sentences = "\n".join(corpus.split('\n')[5:6])
sentences = """
the asbestos fiber <unk> is unusually <unk> once it enters the <unk> 
with even brief exposures to it causing symptoms that show up decades later researchers said
"""
sequences = word_indexing.function(sentences)
for pair in zip(sentences.strip().split(" "), sequences[0]):
    print(f"{pair[0]:15} : {pair[1]:5}")

Sentence is empty. Skipping...
Sentence is empty. Skipping...


the             :  3865
asbestos        :  4861
fiber           :   356
<unk>           :     1
is              :  2643
unusually       :  7375
<unk>           :     1
once            :  5904
it              :  7722
enters          :  7242
the             :  3865
<unk>           :     1

with           :     0
even            :     0
brief           :     0


---
# Context of a word in a sentence

In the sentence ```"a form of asbestos once used to make kent cigarette filters"```, one of the context windows ```a form of asbestos once``` of size 5 and event size 1 has.
* ```of``` as a target word.
* ```(a, form) and (asbestos, once)``` as its context.

### Sequence of the word indices for the sentence

In [13]:
sentences = """
a form of asbestos once used to make kent cigarette filters

N years old and former chairman of consolidated gold fields plc was named a nonexecutive director
"""

sequence = word_indexing.function(sentences)
sequence

Sentence is empty. Skipping...
Sentence is empty. Skipping...
Sentence is empty. Skipping...


array([[2274, 4665, 8174, 4861, 5904, 7477, 8907, 8707,  198, 9119, 3214,    0,    0,    0,    0,    0],
       [5016, 6114, 5416, 5980, 9300, 4650, 8174, 8850, 4103, 5874, 9391, 7317,  406, 2274, 7383, 1620]])

## Target, context pairs

For each word in the setence ```(of, asbestos, ... , kent)``` excludnig the ends of the sentence, create ```(target, context)``` as:

```
[
  [of, a, form, asbestos, once],              # target is 'of', context is (a, form, asbestos, once)
  ['asbestos', 'form', 'of', 'once', 'used'],
  ['once', 'of', 'asbestos', 'used', 'to'],
  ...
]
```

In [14]:
event_context = EventContext(
    name="ev",
    window_size=5,
    event_size=1    
)

In [15]:
event_context_pairs = event_context.function(sequence)
event_context_pairs

array([[[8174, 2274, 4665, 4861, 5904],
        [4861, 4665, 8174, 5904, 7477],
        [5904, 8174, 4861, 7477, 8907],
        [7477, 4861, 5904, 8907, 8707],
        [8907, 5904, 7477, 8707,  198],
        [8707, 7477, 8907,  198, 9119],
        [ 198, 8907, 8707, 9119, 3214],
        [9119, 8707,  198, 3214,    0],
        [3214,  198, 9119,    0,    0],
        [   0, 9119, 3214,    0,    0],
        [   0, 3214,    0,    0,    0],
        [   0,    0,    0,    0,    0]],

       [[5416, 5016, 6114, 5980, 9300],
        [5980, 6114, 5416, 9300, 4650],
        [9300, 5416, 5980, 4650, 8174],
        [4650, 5980, 9300, 8174, 8850],
        [8174, 9300, 4650, 8850, 4103],
        [8850, 4650, 8174, 4103, 5874],
        [4103, 8174, 8850, 5874, 9391],
        [5874, 8850, 4103, 9391, 7317],
        [9391, 4103, 5874, 7317,  406],
        [7317, 5874, 9391,  406, 2274],
        [ 406, 9391, 7317, 2274, 7383],
        [2274, 7317,  406, 7383, 1620]]], dtype=int32)

### Target, context pairs in textual words

In [16]:
word_indexing.sequence_to_sentence(event_context_pairs)

[[['of', 'a', 'form', 'asbestos', 'once'],
  ['asbestos', 'form', 'of', 'once', 'used'],
  ['once', 'of', 'asbestos', 'used', 'to'],
  ['used', 'asbestos', 'once', 'to', 'make'],
  ['to', 'once', 'used', 'make', 'kent'],
  ['make', 'used', 'to', 'kent', 'cigarette'],
  ['kent', 'to', 'make', 'cigarette', 'filters'],
  ['cigarette', 'make', 'kent', 'filters', '<nil>'],
  ['filters', 'kent', 'cigarette', '<nil>', '<nil>'],
  ['<nil>', 'cigarette', 'filters', '<nil>', '<nil>'],
  ['<nil>', 'filters', '<nil>', '<nil>', '<nil>'],
  ['<nil>', '<nil>', '<nil>', '<nil>', '<nil>']],
 [['old', 'n', 'years', 'and', 'former'],
  ['and', 'years', 'old', 'former', 'chairman'],
  ['former', 'old', 'and', 'chairman', 'of'],
  ['chairman', 'and', 'former', 'of', 'consolidated'],
  ['of', 'former', 'chairman', 'consolidated', 'gold'],
  ['consolidated', 'chairman', 'of', 'gold', 'fields'],
  ['gold', 'of', 'consolidated', 'fields', 'plc'],
  ['fields', 'consolidated', 'gold', 'plc', 'was'],
  ['plc', 'g